# Laboratorium 6 

# Piotr Rzeźnik 

In [12]:
(<$<) :: (a -> b) -> a -> b
(<$<) = ($)

(>$>) :: a -> (a -> b) -> b
x >$> f = f x
infixl 0 >$>

In [13]:
(\x -> 2 * x + 1) <$< 1
1 >$> (\x -> 2 * x + 1)

3

3

In [14]:
(<.<) :: (b -> c) -> (a -> b) -> (a -> c)
(<.<) = (.)

(>.>) :: (a -> b) -> (b -> c) -> (a -> c)
f >.> g = g . f
infixl 9 >.>

In [15]:
(+2) <.< (*3) <$< 5
(+2) >.> (*3) <$< 5

17

21

In [16]:
5 >$> (+2) <.< (*3)
5 >$> (+2) >.> (*3)

17

21

In [17]:
3 >$> (+2) >$> (+10)

15

In [18]:
tail [1,2]
tail >.> tail >.> tail $ [1,2]

[2]

: 

In [19]:
safeTail :: [a] -> Maybe [a]
safeTail []     = Nothing
safeTail (x:xs) = Just xs

In [20]:
safeTail [1,2]
safeTail []

Just [2]

Nothing

In [21]:
safeTail >.> safeTail >.> safeTail <$< [1,2]

: 

In [22]:
[1,2] >$> safeTail >.> safeTail >.> safeTail

: 

In [23]:
:t safeTail

safeTail :: forall a. [a] -> Maybe [a]

In [24]:
extractMaybe :: Maybe a -> a
extractMaybe Nothing  = error "Nothing inside!"
extractMaybe (Just x) = x

In [25]:
[1,2] >$> safeTail >.> extractMaybe >.> safeTail
[1,2] >$> safeTail >.> extractMaybe >.> safeTail >.> extractMaybe >.> safeTail 

Just []

Nothing

In [26]:
[1] >$> safeTail >.> extractMaybe >.> safeTail >.> extractMaybe >.> safeTail

: 

In [27]:
insertMaybe :: a -> Maybe a
insertMaybe = Just

-- (>^$>) = extract (^) and apply ($)
(>^$>) :: Maybe a -> (a -> Maybe b) -> Maybe b
ma >^$> f = (extractMaybe ma) >$> f
infixl 1 >^$>

Line 6: Redundant bracket
Found:
(extractMaybe ma) >$> f
Why not:
extractMaybe ma >$> f

In [28]:
insertMaybe [1,2]
[1,2] >$> insertMaybe >.> extractMaybe

Just [1,2]

[1,2]

In [29]:
[1,2] >^$> safeTail


: 

In [30]:
:t (>^$>)
insertMaybe [1,2] >^$> safeTail
insertMaybe [1,2] >^$> safeTail >^$> safeTail
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail

(>^$>) :: forall a b. Maybe a -> (a -> Maybe b) -> Maybe b

Just [2]

Just []

Nothing

: 

In [31]:
(>^$>) :: Maybe a -> (a -> Maybe b) -> Maybe b
Nothing  >^$> _ = Nothing
(Just x) >^$> f = f x
infixl 1 >^$>

In [32]:
insertMaybe [1,2] >^$> safeTail
insertMaybe [1,2] >^$> safeTail >^$> safeTail
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail

Just [2]

Just []

Nothing

Nothing

In [33]:
f1 :: (Ord a, Num a) => a -> Maybe a
f1 x = if x > 0 then Just (x + 1) else Nothing

f2 :: (Eq a, Num a) => a -> Maybe a
f2 x = if x /= 0 then Just (10 * x) else Nothing

-- Kleisli composition
(>.>>) :: (a -> Maybe b) -> (b -> Maybe c) -> (a -> Maybe c)
f >.>> g = \x -> g (extractMaybe (f x))

Line 9: Avoid lambda
Found:
\ x -> g (extractMaybe (f x))
Why not:
g . extractMaybe . f

In [34]:
h = f1 >.>> f2
:t h
f1 3
f2 3
h 3

h :: forall c. (Ord c, Num c) => c -> Maybe c

Just 4

Just 30

Just 40

In [35]:
let f3 x = Just (1/x)
let f4 x = Just (2 * x)
:t f3
:t f4

let f34 = f3 >.>> f4
:t f34
f34 2

f3 :: forall a. Fractional a => a -> Maybe a

f4 :: forall a. Num a => a -> Maybe a

f34 :: forall c. Fractional c => c -> Maybe c

Just 1.0

### 2


In [36]:
:i Maybe
:i Monad

return 1 :: Maybe Int

Just 1

In [37]:
:t (>=>)


(>=>) :: forall (m :: * -> *) a b c. Monad m => (a -> m b) -> (b -> m c) -> a -> m c

In [38]:
import Control.Monad
:t (>=>)

(>=>) :: forall (m :: * -> *) a b c. Monad m => (a -> m b) -> (b -> m c) -> a -> m c

In [39]:
insertMaybe [1,2]



Just [1,2]

In [40]:
return (Just [1,2])
return [1,2]
return [1,2] :: Maybe [Int]


Just [1,2]

[1,2]

Just [1,2]

In [41]:
insertMaybe [1,2] >^$> safeTail
return [1,2] >>= safeTail
(return [1,2] >>= safeTail) == (insertMaybe [1,2] >^$> safeTail)



Line 2: Monad law, left identity
Found:
return [1, 2] >>= safeTail
Why not:
safeTail [1, 2]Line 3: Monad law, left identity
Found:
return [1, 2] >>= safeTail
Why not:
safeTail [1, 2]

Just [2]

Just [2]

True

In [42]:
return [1,2] >>= safeTail


Line 1: Monad law, left identity
Found:
return [1, 2] >>= safeTail
Why not:
safeTail [1, 2]

Just [2]

In [43]:
(return [1,2] >>= safeTail) == (insertMaybe [1,2] >^$> safeTail)

Line 1: Monad law, left identity
Found:
return [1, 2] >>= safeTail
Why not:
safeTail [1, 2]

True

In [44]:
insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail
return [1,2] >>= safeTail >>= safeTail >>= safeTail >>= safeTail


Line 2: Monad law, left identity
Found:
return [1, 2] >>= safeTail
Why not:
safeTail [1, 2]

Nothing

Nothing

In [45]:
return [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail
insertMaybe [1,2] >>= safeTail >^$> safeTail >>= safeTail >^$> safeTail
return [1,2] >^$> safeTail >>= safeTail >^$> safeTail >>= safeTail

Nothing

Nothing

Nothing

In [46]:
doSafeTail3x :: [a] -> Maybe [a]
doSafeTail3x xs = do
  t1 <- safeTail xs
  t2 <- safeTail t1
  t3 <- safeTail t2
  return t3

safeTail3x :: [a] -> Maybe [a]
safeTail3x xs =
  safeTail xs >>= \t1 ->
    safeTail t1 >>= \t2 ->
      safeTail t2 >>= \t3 ->
        return t3

safeTail3x' :: [a] -> Maybe [a]
safeTail3x' xs = return xs >>= safeTail >>= safeTail >>= safeTail

Line 2: Redundant return
Found:
do t1 <- safeTail xs
   t2 <- safeTail t1
   t3 <- safeTail t2
   return t3
Why not:
do t1 <- safeTail xs
   t2 <- safeTail t1
   safeTail t2Line 10: Use >=>
Found:
\ t1 -> safeTail t1 >>= \ t2 -> safeTail t2 >>= \ t3 -> return t3
Why not:
(safeTail Control.Monad.>=>
   (\ t2 -> safeTail t2 >>= \ t3 -> return t3))Line 11: Use >=>
Found:
\ t2 -> safeTail t2 >>= \ t3 -> return t3
Why not:
(safeTail Control.Monad.>=> (\ t3 -> return t3))Line 16: Monad law, left identity
Found:
return xs >>= safeTail
Why not:
safeTail xs

In [47]:
doSafeTail3x [1..5]
safeTail3x [1..5]
safeTail3x' [1..5]

Just [4,5]

Just [4,5]

Just [4,5]

In [48]:
doSafeTail3x [1]
safeTail3x [1]
safeTail3x' [1]

Nothing

Nothing

Nothing

In [49]:
f5 :: Int -> Int -> Int -> Int
f5 x y z = 1000 `div` x + 100 `div` y + 10 `div` z

In [50]:
f5 1 2 3
f5 1 2 0


1053

: 

In [51]:
f5 1 0 3


: 

In [52]:
f5 0 2 3

: 

In [53]:
safeDiv :: Int -> Int -> Maybe Int
safeDiv x y | y /= 0    = Just $ x `div` y
            | otherwise = Nothing

safeF5 :: Int -> Int -> Int -> Maybe Int
safeF5 x y z =
  case (safeDiv 1000 x) of
    Nothing -> Nothing
    Just (iOverX) ->
      case (safeDiv 100 y) of
        Nothing -> Nothing
        Just (iOverY) ->
          case (safeDiv 10 z) of
            Nothing -> Nothing
            Just (iOverZ) -> Just $ iOverX + iOverY + iOverZ

Line 7: Redundant bracket
Found:
case (safeDiv 1000 x) of
    Nothing -> Nothing
    Just (iOverX) -> case (safeDiv 100 y) of
                         Nothing -> Nothing
                         Just (iOverY) -> case (safeDiv 10 z) of
                                              Nothing -> Nothing
                                              Just (iOverZ) -> Just $ iOverX + iOverY + iOverZ
Why not:
case safeDiv 1000 x of
    Nothing -> Nothing
    Just (iOverX) -> case (safeDiv 100 y) of
                         Nothing -> Nothing
                         Just (iOverY) -> case (safeDiv 10 z) of
                                              Nothing -> Nothing
                                              Just (iOverZ) -> Just $ iOverX + iOverY + iOverZLine 9: Redundant bracket
Found:
(iOverX)
Why not:
iOverXLine 10: Redundant bracket
Found:
case (safeDiv 100 y) of
    Nothing -> Nothing
    Just (iOverY) -> case (safeDiv 10 z) of
                         Nothing -> Nothing
                         Just (iOverZ) -> Just $ iOverX + iOverY + iOverZ
Why not:
case safeDiv 100 y of
    Nothing -> Nothing
    Just (iOverY) -> case (safeDiv 10 z) of
                         Nothing -> Nothing
                         Just (iOverZ) -> Just $ iOverX + iOverY + iOverZLine 12: Redundant bracket
Found:
(iOverY)
Why not:
iOverYLine 13: Redundant bracket
Found:
case (safeDiv 10 z) of
    Nothing -> Nothing
    Just (iOverZ) -> Just $ iOverX + iOverY + iOverZ
Why not:
case safeDiv 10 z of
    Nothing -> Nothing
    Just (iOverZ) -> Just $ iOverX + iOverY + iOverZLine 15: Redundant bracket
Found:
(iOverZ)
Why not:
iOverZ

In [54]:
safeF5 1 2 3
safeF5 1 2 0
safeF5 1 0 3
safeF5 0 2 3

Just 1053

Nothing

Nothing

Nothing

In [55]:
safeF5' :: Int -> Int -> Int -> Maybe Int
safeF5' x y z = do
  iOverX <- safeDiv 1000 x
  iOverY <- safeDiv 100 y
  iOverZ <- safeDiv 10 z
  return $ iOverX + iOverY + iOverZ

In [56]:
safeF5' 1 2 3
safeF5' 1 2 0
safeF5' 1 0 3
safeF5' 0 2 3

Just 1053

Nothing

Nothing

Nothing

### 3.

In [57]:
:i Either
:i Monad

return 1 :: Either String Int

Right 1

In [58]:
safeTail :: [a] -> Either String [a]
safeTail []     = Left "Empty list!"
safeTail (x:xs) = Right xs

In [59]:
safeTail [1..4]
safeTail [1]
safeTail []

Right [2,3,4]

Right []

Left "Empty list!"

In [60]:
doSafeTail3x :: [a] -> Either String [a]
doSafeTail3x xs = do
  t1 <- safeTail xs
  t2 <- safeTail t1
  t3 <- safeTail t2
  return t3

Line 2: Redundant return
Found:
do t1 <- safeTail xs
   t2 <- safeTail t1
   t3 <- safeTail t2
   return t3
Why not:
do t1 <- safeTail xs
   t2 <- safeTail t1
   safeTail t2

In [61]:
doSafeTail3x [1..4]
doSafeTail3x [1..3]
doSafeTail3x [1,2]

Right [4]

Right []

Left "Empty list!"

In [62]:
safeDiv :: Int -> Int -> Either String Int
safeDiv x y | y /= 0    = Right $ x `div` y
            | otherwise = Left "Cannot div by zero!"

safeF5 :: Int -> Int -> Int -> Either String Int
safeF5 x y z =
  case (safeDiv 1000 x) of
    Left e -> Left e
    Right (iOverX) ->
      case (safeDiv 100 y) of
        Left e -> Left e
        Right (iOverY) ->
          case (safeDiv 10 z) of
            Left e -> Left e
            Right (iOverZ) -> Right $ iOverX + iOverY + iOverZ

Line 7: Redundant bracket
Found:
case (safeDiv 1000 x) of
    Left e -> Left e
    Right (iOverX) -> case (safeDiv 100 y) of
                          Left e -> Left e
                          Right (iOverY) -> case (safeDiv 10 z) of
                                                Left e -> Left e
                                                Right (iOverZ) -> Right $ iOverX + iOverY + iOverZ
Why not:
case safeDiv 1000 x of
    Left e -> Left e
    Right (iOverX) -> case (safeDiv 100 y) of
                          Left e -> Left e
                          Right (iOverY) -> case (safeDiv 10 z) of
                                                Left e -> Left e
                                                Right (iOverZ) -> Right $ iOverX + iOverY + iOverZLine 9: Redundant bracket
Found:
(iOverX)
Why not:
iOverXLine 10: Redundant bracket
Found:
case (safeDiv 100 y) of
    Left e -> Left e
    Right (iOverY) -> case (safeDiv 10 z) of
                          Left e -> Left e
                          Right (iOverZ) -> Right $ iOverX + iOverY + iOverZ
Why not:
case safeDiv 100 y of
    Left e -> Left e
    Right (iOverY) -> case (safeDiv 10 z) of
                          Left e -> Left e
                          Right (iOverZ) -> Right $ iOverX + iOverY + iOverZLine 12: Redundant bracket
Found:
(iOverY)
Why not:
iOverYLine 13: Redundant bracket
Found:
case (safeDiv 10 z) of
    Left e -> Left e
    Right (iOverZ) -> Right $ iOverX + iOverY + iOverZ
Why not:
case safeDiv 10 z of
    Left e -> Left e
    Right (iOverZ) -> Right $ iOverX + iOverY + iOverZLine 15: Redundant bracket
Found:
(iOverZ)
Why not:
iOverZ

In [63]:
safeF5 1 2 3
safeF5 1 2 0
safeF5 1 0 3
safeF5 0 2 3

Right 1053

Left "Cannot div by zero!"

Left "Cannot div by zero!"

Left "Cannot div by zero!"

In [64]:
safeF5' :: Int -> Int -> Int -> Either String Int
safeF5' x y z = do
  iOverX <- safeDiv 1000 x
  iOverY <- safeDiv 100 y
  iOverZ <- safeDiv 10 z
  return $ iOverX + iOverY + iOverZ

In [65]:
safeF5' 1 2 3
safeF5' 1 2 0
safeF5' 1 0 3
safeF5' 0 2 3

Right 1053

Left "Cannot div by zero!"

Left "Cannot div by zero!"

Left "Cannot div by zero!"

### 4.

In [66]:
:i []

[1,2,3] :: [Int]
[1,2,3] :: [] Int

[1,2,3]

[1,2,3]

In [68]:
return 1 :: [] Int
return 3 >>= (\x -> [1..x])
[1,2] >>= (\x -> [-x,x])
[1,2,3] >>= (\x -> [-x..x])



[1]

[1,2,3]

[-1,1,-2,2]

[-1,0,1,-2,-1,0,1,2,-3,-2,-1,0,1,2,3]

In [69]:
[1,2] >>= (\x -> [-x,x]) >>= (\y -> [-y,y])
[1,2] >>= \x -> [-x,x] >>= \y -> [-y,y]
[1,2] >>= \x -> [-x,x] >>= \y -> return (x,y)

[1,-1,-1,1,2,-2,-2,2]

[1,-1,-1,1,2,-2,-2,2]

[(1,-1),(1,1),(2,-2),(2,2)]

In [70]:
xs1 :: [(Int,Int,Int)]
xs1 = [ (x,y,z) | let xs = [1,2], x <- xs, y <- xs, z <- xs ]

doXs1 :: [(Int,Int,Int)]
doXs1 = do
  let xs = [1,2]
  x <- xs
  y <- xs
  z <- xs
  return (x,y,z)

In [71]:
xs1
doXs1
xs1 == doXs1

[(1,1,1),(1,1,2),(1,2,1),(1,2,2),(2,1,1),(2,1,2),(2,2,1),(2,2,2)]

[(1,1,1),(1,1,2),(1,2,1),(1,2,2),(2,1,1),(2,1,2),(2,2,1),(2,2,2)]

True

In [72]:
import Control.Monad

In [73]:
xs2 :: [(Int,Int,Int)]
xs2 = [ (x,y,z) | let xs = [1..3], x <- xs, y <- xs, z <- xs, x > y && y > z ]

doXs2 :: [(Int,Int,Int)]
doXs2 = do
  let xs = [1..3]
  x <- xs
  y <- xs
  z <- xs
  guard $ x > y && y > z
  return (x,y,z)

doXs2' :: [(Int,Int,Int)]
doXs2' = do
  let xs = [1..3]
  x <- xs
  y <- xs
  z <- xs
  if x > y && y > z
    then return (x,y,z)
    else []

Line 6: Reduce duplication
Found:
let xs = [1 .. 3]
x <- xs
y <- xs
z <- xs

Why not:
Combine with -:15:3

In [74]:
xs2
doXs2
doXs2'

[(3,2,1)]

[(3,2,1)]

[(3,2,1)]

In [76]:
xs2 == doXs2
xs2 == doXs2'
doXs2 == doXs2'

True

True

True

### 5

In [77]:
:i Writer


: 

In [78]:
import Control.Monad.Trans.Writer.Lazy
:i Writer
:i WriterT

In [79]:
:t writer
:t runWriter
:t execWriter
:t tell



writer :: forall (m :: * -> *) a w. Monad m => (a, w) -> WriterT w m a

runWriter :: forall w a. Writer w a -> (a, w)

execWriter :: forall w a. Writer w a -> w

tell :: forall (m :: * -> *) w. Monad m => w -> WriterT w m ()

In [80]:
runWriter $ writer(1,["op1","op2"])
execWriter $ writer(1,["op1","op2"])


(1,["op1","op2"])

["op1","op2"]

In [81]:

runWriter (return 1 :: Writer String Int)
runWriter (return [1] :: Writer String [Int])
runWriter (return (1,1) :: Writer [String] (Int,Int))

(1,"")

([1],"")

((1,1),[])

In [82]:
import Control.Monad.Trans.Writer.Lazy

gcdWithLog :: Int -> Int -> Writer [String] Int
gcdWithLog a b
  | b == 0 = do
      tell ["Finished with " ++ show a]
      return a
  | otherwise = do
      tell [show a ++ " mod " ++ show b ++ " = " ++ show (a `mod` b)]
      gcdWithLog b (a `mod` b)

In [83]:
gcdWithLog 4 16
runWriter $ gcdWithLog 4 16
execWriter $ gcdWithLog 4 16

WriterT (Identity (4,["4 mod 16 = 4","16 mod 4 = 0","Finished with 4"]))

(4,["4 mod 16 = 4","16 mod 4 = 0","Finished with 4"])

["4 mod 16 = 4","16 mod 4 = 0","Finished with 4"]

In [84]:
mapWithLog :: Show a => (a -> b) -> [a] -> Writer [String] [b]
mapWithLog _ [] = do
  tell ["map []"]
  return []
mapWithLog f (x:xs) = do
  tell ["map " ++ show x]
  mapXs <- mapWithLog f xs
  return $ f x : mapXs

In [85]:
mapWithLog (*2) [1..5]
runWriter $ mapWithLog (*2) [1..5]
execWriter $ mapWithLog (*2) [1..5]

WriterT (Identity ([2,4,6,8,10],["map 1","map 2","map 3","map 4","map 5","map []"]))

([2,4,6,8,10],["map 1","map 2","map 3","map 4","map 5","map []"])

["map 1","map 2","map 3","map 4","map 5","map []"]